In [1294]:
'''
Replace U1 with quantum gates instead of matrix
'''
from qiskit import transpile, QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.circuit.library.standard_gates import XGate
from qiskit.providers.basic_provider import BasicSimulator

# Discrete Quantum Random Walks On The Line

This is a notebook about random walks on the line


## quantum version of the unitary matrix


### initialize_state

this function maps an integer number from bits to qubits

for 3 bits n=3
000 -> |000>
001 -> |001>
...
2^n -> |2^n>

for n bits we have 2^n different states

In [1295]:
def initialize_state(qc, digits):
    for i, digit in enumerate(reversed(digits)):
        if digit == '1':
            qc.x(i)

### Input


In [1296]:
num = "00001"

### Circuit Initialization
The Quantum circuit has quantum registers to execute the algorithm
and classical registers to measure the results

In [1297]:
num_of_qubits = len(num)
cr = ClassicalRegister(num_of_qubits)
qr = QuantumRegister(num_of_qubits)
qc = QuantumCircuit(qr, cr)

### State initialization

In [1298]:
initialize_state(qc, num)

In [1299]:
def transition_state(qc, num, ctrl_state=None):
    for target_qubit in range(len(num) - 1, 0, -1):
        qargs = [control_qubit for control_qubit in range(target_qubit)]
        qargs.append(target_qubit)
        cx_gate = XGate().control(target_qubit, ctrl_state=ctrl_state)
        qc.append(cx_gate, qargs)
    qc.x(0)

In [1300]:
# positive transition
transition_state(qc, num)

# # negative transition
# transition_state(qc, num, ctrl_state=0)

In [1305]:
# qc.decompose().draw()

┌───────────┐                                                      »
q2364_0: ┤ U3(π,0,π) ├──■────■─────────────────────■─────────────────────■──»
         └───────────┘  │    │                     │             ┌───┐   │  »
q2364_1: ───────────────■────■─────────■───────────┼─────────■───┤ T ├───┼──»
                        │    │  ┌───┐┌─┴─┐┌─────┐┌─┴─┐┌───┐┌─┴─┐┌┴───┴┐┌─┴─┐»
q2364_2: ───────────────■────■──┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├»
                        │  ┌─┴─┐└┬─┬┘└───┘└─┬─┬─┘└───┘└───┘└───┘└─────┘└───┘»
q2364_3: ───────────────■──┤ X ├─┤M├────────┤M├─────────────────────────────»
                      ┌─┴─┐└┬─┬┘ └╥┘  ┌─┐   └╥┘                             »
q2364_4: ─────────────┤ X ├─┤M├───╫───┤M├────╫──────────────────────────────»
                      └───┘ └╥┘   ║   └╥┘    ║                              »
 c186: 5/════════════════════╩════╩════╩═════╩══════════════════════════════»
                             4    3    4     3                              »
«               ┌───┐           ┌───────────┐┌─┐   ┌─┐
«q2364_0: ──■───┤ T ├───■────■──┤ U3(π,0,π) ├┤M├───┤M├
«         ┌─┴─┐┌┴───┴┐┌─┴─┐┌─┴─┐└────┬─┬────┘└╥┘┌─┐└╥┘
«q2364_1: ┤ X ├┤ Tdg ├┤ X ├┤ X ├─────┤M├──────╫─┤M├─╫─
«         ├───┤└┬───┬┘└┬─┬┘└┬─┬┘     └╥┘      ║ └╥┘ ║ 
«q2364_2: ┤ T ├─┤ H ├──┤M├──┤M├───────╫───────╫──╫──╫─
«         └───┘ └───┘  └╥┘  └╥┘       ║       ║  ║  ║ 
«q2364_3: ──────────────╫────╫────────╫───────╫──╫──╫─
«                       ║    ║        ║       ║  ║  ║ 
«q2364_4: ──────────────╫────╫────────╫───────╫──╫──╫─
«                       ║    ║        ║       ║  ║  ║ 
« c186: 5/══════════════╩════╩════════╩═══════╩══╩══╩═
«                       2    2        1       0  1  0

In [1306]:
qubits = list(range(num_of_qubits))
qc.measure(qubits, qubits)

backend = BasicSimulator()
tqc = transpile(qc, backend)
counts = backend.run(tqc).result().get_counts()
print(counts)

{'00010': 1024}
